<!-- vscode-jupyter-toc -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
<a id='toc0_'></a>**Содержание**    
- [Ключевые слова static и inline](#toc1_)    
  - [Глобальные переменные (extern)](#toc1_1_)    
  - [Статические глобальные переменные (static)](#toc1_2_)    
  - [Статические локальные переменные](#toc1_3_)    
  - [Статические функции](#toc1_4_)    
  - [Статические поля класса](#toc1_5_)    
  - [Статические методы](#toc1_6_)    
  - [Ключевое слово inline](#toc1_7_)    
    - [Тут нужно отступление про встраивание из вики](#toc1_7_1_)    
  - [Правило одного определения](#toc1_8_)    
- [Ключевое слово friend](#toc2_)    
  - [Дружественные классы](#toc2_1_)    
  - [Дружественные функции](#toc2_2_)    
  - [Дружественные методы](#toc2_3_)    
  - [Отношение дружбы](#toc2_4_)    
- [Шаблон проектирования Singleton](#toc3_)    
  - [Класс Singleton](#toc3_1_)    
  - [Использование Singleton-а](#toc3_2_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- /vscode-jupyter-toc -->

# <a id='toc1_'></a>[Ключевые слова static и inline](#toc0_)

## <a id='toc1_1_'></a>[Глобальные переменные (extern)](#toc0_)
 
Это такие переменные, которые объявлены (и определены) не в функции.  

Объявление глобальной переменной (объявленим м.б. несколько (чтоб оно дошло до тех, до кого необходимо)):

    // удобно и принято размещать в заголовочных файлах .hpp
    extern  int  global;    // тип int имя global
    
    void f () {
        ++ global;
    }

Определение глобальной переменной (определение одно). В какой единице трансляции \*) определена глобальная переменна, там и будет выделено для нее место. Если несколько определений - ошибка линковки:
    
    // в исходном коде .cpp 
    int  global = 10;       // вне функций
    // это же кстати считается объявлением и определением, если стоит вне функции

Время жизни - от запуска до выхода из программы.  

Проблемы глобальных переменных:

    ∙Масштабируемость. Проблемы зависимости от одной переменной нескольких сущностей (базы данных, потоки выполнения и др.) 
    - нужно синхронизировать и костылять.
    ∙Побочные эффекты. Могут добавлять неопределенность, если от них зависит значение функции, а менять их может кто угодно.
    ∙Порядок инициализации. В одной единице трансляции иниц. последовательно сверху вниз, но если в разных, то 
    стандарт не регулирует это, а значит, если глобальные перем. зависят друг от друга, то UB.
    ...

Стоит их избегать и минимизировать. Но можно в них захардкодить например физ.константы, или флаги командной строки...

\*) "единица трансляции" это файл c/cpp после включения всех его заголовков (.h/.hpp) и обработки препроцессором. Иногда называют модулем.

## <a id='toc1_2_'></a>[Статические глобальные переменные (static)](#toc0_)

\*) static имеет 5 значений в С++. 

Статическая глобальная переменная — это глобальная переменная, доступная только в пределах модуля (единицы трансляции).  
Можно в другом модуле использовать такое же имя.  
=> не может быть завязана на другую перем. из другого модуля, меньше проблем.
=> остается проблема, если она завязана на глобальную, у которой может быть пролблемы.

Определение:  

    static  int  global = 10;  //
    
    void f () {
        ++ global;
    }

Проблемы статических глобальных переменных:

    ∙Масштабируемость.
    ∙Побочные эффекты.

Как и многого другого в С++, их также стоит избегать.

## <a id='toc1_3_'></a>[Статические локальные переменные](#toc0_)

Статическая локальная переменная — это глобальная переменная, доступная только в пределах функции.

Время жизни такой переменной — от первого вызова функции next до конца программы.   
Хранится не на стеке, а вместе в другими глобальными переменными.  
Инициализируется только при первом вызове функции.  

    int  next(int  start = 0) {
        static  int k = start;
        return k++;
    }

    next(10);   // 10 инициализация состоялась
    next(20);   // 11, аргумент ингнорится

Проблемы статических локальных переменных:

    ∙Масштабируемость.
    ∙Побочные эффекты.

## <a id='toc1_4_'></a>[Статические функции](#toc0_)

Статическая функция, доступная только в пределах модуля. Т.е. ведет себя как статическая глобальная переменная (обычная фукнция ведет себя как глобальная переменная, т.е. доступна во всех единицах трансляции, где есть ее объявление).

=> можно определить одинаковые (по имени и сигнатуре) стат. функции в разных модулях, ошибки линковки не будет.

Файл1.cpp:

    static  void  test() {
        cout  << "A\n";
    }

Файл2.cpp:

    static  void  test() {
        cout  << "B\n";
    }

Поэтому говорят, что статические глобальные переменные и статические функции проходят внутреннюю линковку. В отличие от глобальных перем. и обычных функций, которые проходят внешнуюю линковку (отсюда и слово extern). Можно и функции приписать extern в объявлении, но это не обязательно.

## <a id='toc1_5_'></a>[Статические поля класса](#toc0_)

Статические поля класса — это глобальные переменные, определённые внутри класса.  
Хранятся в области где и глобальные переменные (в области глобальных данных), создаются при запуске и удаляются при завершении программы. Т.е. полностью тоже самое за исключением модификаторов доступа. 

Объявление:
    
    // посчитаем юзеров (экземпляры класса) без использования глобальной переменной, считающей вызовы конструкторов/деструкторов.
    
    struct  User {
        ...
    private:                            // приватная - кто попало менять не может, только методы этого класса
        static  size_t  instances_;     // объявили статическое поле (живет даже когда экземпляров нет)
    };

Определение:
    
    size_t  User::instances_ = 0;       // снаружи класса дали определение в котором захрардкодили стартовое значение, а методы пусть плюсуют / минусуют
                                        // в каком файле с кодом определили (обязательно вне функций), в том и будет жить переменная
    // *в классе запрещено инициализировать неконстантные статические поля (не статические - можно)

Обычные поля - это поля объекта. А это - поле класса.  
Для доступа к статическим полям не нужен объект, только тип.  

## <a id='toc1_6_'></a>[Статические методы](#toc0_)

Статические методы — это функции, определённые внутри класса и имеющие доступ к закрытым полям и методам.
Плюс на них в отличие от обычных функций, действуют модификаторы доступа (доступа к ним конечно же).

Нет this (но и нет объекта класса cls, т.к. вообще нет объектов классов в С++ ?).

Объявление:

    struct  User {
        ...
        static  size_t  count () { return  instances_; }
    private:
        static  size_t  instances_;
    };

Для вызова статических методов не нужен объект.
    
    cout  << User::count ();

*) Т.к. это метод класса, то можно в него передать объект данного класса и получить доступ к его закрытым полям.

## <a id='toc1_7_'></a>[Ключевое слово inline](#toc0_)

Тут нет многозначности как в static.

Советует компилятору встроить данную функцию.

    inline  double  square(double x) { return x * x; }

∙В месте вызова inline-функции должно быть известно её определение. Необходимое условие, чтобы компилятор имел возможность попытаться ее встроить в код.  
∙inline функции можно (и стоит так делать) определять в заголовочных файлах. Чтобы компилятор, если сможет, встроил их в код по максимуму.  
∙Все методы, определённые внутри класса, являются inline. ТАК ВОТ ПОЧЕМУ методы класса в ассемблерном коде отсутствовали.  
∙При линковке из всех версий inline-функции (т.е. её код из разных единиц трансляции) выбирается только одна.  
∙Все определения одной и той же inline-функции должны быть идентичными. Компилятор предполагает, что они все идентичны, поэтому к этому и надо стремиться. Но он не будет ругаться, это ответственность разработчика?   
∙inline — это совет компилятору, а не указ. Поэтому ДОСТАТОЧНОГО условия не существует.

*) обычный вызов функции (call в бинарном файле) - это множество накладных расходов, подготовить переменные, положить на стек, развернуть, удалить. На простых функциях типа приведенной, они превысят расходы на ее выполнение. 

Применение инлайн фукнций - основное это запулить все (объявление и определение в заголовочный файл), что позволяет создавать большие библиотеки ТОЛЬКО из заголовочных файлов (плюс заглушка для кода с перечислением инклюдов). У таких библиотек есть проблемы - основная это время компиляции, каждая функция компилируется по много раз, выбирается будет ли и какая из них использоваться, все это очень долго.

inline не гарантирует встраивание функции компилятором, поэтому любой вызов потенциально может быть "настоящим" вызовом функции.



### <a id='toc1_7_1_'></a>[Тут нужно отступление про встраивание из вики](#toc0_)

Это способ оптимизации, при котором вызов функции заменяется непосредственно её телом. Встраивание функций аналогично по сути подстановке из макроса, но в отличие от неё не изменяет исходный код и происходит во время компиляции, в то время как макросы изменяют исходный код перед компиляцией.  
Как правило, некоторое количество встраиваний помогает улучшить быстродействие при очень незначительных затратах пространства, но избыточность использования может замедлить программу из-за того, что встроенный код станет потреблять слишком много кэша инструкций, а также займёт значительное количество пространства.

По сути компилятор помещает новую копию функции в каждое место, где она вызывается. Код со встроенными функциями работает немного быстрее обычного, поскольку пропадают дополнительные действия по вызову функций, однако же при этом увеличивается расход памяти. Если функция встроена 10 раз, то в код будет вставлено 10 копий функции. Поэтому встраивание лучше всего подходит для небольших часто используемых функций.

В императивных языках более низкого уровня, таких как C и Fortran, можно достичь увеличения скорости на 10-20% с незначительным влиянием на размер кода. В более абстрактных языках эффект может быть значительно выше из-за количества слоев, которые удаляет встраивание.  

Непосредственная выгода от устранения вызова функции заключается в следующем:

    Устраняются инструкции, необходимые для вызова функции, как в вызывающей функции, так и в вызываемом объекте. 
    Это код для размещения аргументов в стеке или в регистрах, сам вызов функции, пролог функции, в конце тела эпилог функции, 
    оператор возврата[en], извлечение возвращаемого значения, удаление аргументов из стеков и (при необходимости) 
    восстановление регистров.
    Упрощается задача распределения регистров поскольку не нужно задействовать регистры для передачи аргументов .
    Устраняется необходимость передавать ссылки, а затем разыменовывать их при вызовах по ссылке (или при вызовах по соиспользованию).

Однако определение точки безубыточности, за которой встраивание снижает производительность, является сложным и зависит от реальной нагрузки. 

Обычно алгоритму встраивания предоставляется определенный бюджет пространства (допустимое увеличение размера программы). Алгоритм стремится встроить наиболее ценные вызовы, не превысив этого бюджета. В этом смысле многие алгоритмы встраивания обычно моделируют решение задачи о рюкзаке. Для оценки весов используется профилирование.  

Кроме данных профилирования, современные динамические компиляторы применяют несколько более продвинутых эвристик, таких как:

    Оценка того, какие пути исполнения приведут к наибольшему росту быстродействия (при включении дополнительных оптимизаций компилятора после встраивания) и увеличение уровня выгоды от таких путей для алгоритма.
    Адаптивная корректировка порога безубыточности встраивания на основе размера единицы компиляции и объема уже встроенного кода.
    Группировка подпрограмм в кластеры и встраивание целых кластеров вместо отдельных подпрограмм. В этом случае эвристика подбирает кластеры, группируя методы, для которых встраивание правильного подмножества кластера приводит к худшей производительности, чем отсутствие встраивания.

Полное встраивание не всегда возможно из-за рекурсии: рекурсивное встраивание вызовов никогда не завершится.

## <a id='toc1_8_'></a>[Правило одного определения](#toc0_)

Правило одного определения (One Definition Rule, ODR)

∙В пределах любой единицы трансляции сущности не могут иметь более одного определения. Объявлений может быть много. Ошибки такого рода выявляет этапе компиляции.     
∙В пределах программы глобальные переменные и не-inline функции не могут иметь больше одного определения.  Ошибки такого рода выявляет этапе линковки.  
∙Классы и inline функции могут определяться в более чем одной единице трансляции, но определения обязаны совпадать. Ошибки такого рода никто не выявляет, они просто ведут к неопределенному поведению.  

Вопрос: к каким проблемам могут привести разные определения одного класса в разных частях программы?

    методы - это инлайн функции, будут выбраны какие попало компилятором, больше ничего страшного не произойдет - UB

Вот эту мысль преподавателя надо еще подумать:   
т.е. чтобы использовать некоторый класс в разных единицах трансляции, то неизбежно он должен быть определён в каждой из единиц трансляции, в которой он используется. Для этого его определение переносят в заголовочный файл и подключают в каждый cpp файл, в котором этот класс нужен.

Определение класса - это просто список полей и методов), а в cpp файл выносятся только определения методов.

Class declaration

    ...
    lass-key 	    - 	one of class or struct.
    attr(C++11) 	- 	optional sequence of any number of attributes, may include alignas specifier
    class-head-name - 	the name of the class that's being defined.
    base-clause 	- 	optional list of one or more parent classes and the model of inheritance used for each

    body of a class definition:
    member-specification - list of access specifiers, member object and member function declarations and DEFINITIONS

Forward declaration

Declares a class type which will be defined later in this scope. Until the definition appears, this class name has incomplete type. This allows classes that refer to each other...


# <a id='toc2_'></a>[Ключевое слово friend](#toc0_)

## <a id='toc2_1_'></a>[Дружественные классы](#toc0_)

    struct  String {
        ...
        friend  struct  StringBuffer;       // объявляем friend класс (в каком блоке объявим - не важно)
    private:                                // не хотим раскрывать поля через публичный и защищенный интерфейс
        char * data_;
        size_t  len_;
    };
    
    struct  StringBuffer {
        void  append(String  const& s) {
            append(s.data_);                // можем обращаться к приватным полям извне класса, как будто мы члены класса
        }
        void  append(char  const* s) {...}
        ...
    };

## <a id='toc2_2_'></a>[Дружественные функции](#toc0_)

Дружественные функции можно определять прямо внутри описания класса (они становятся inline).

    struct  String {
        ...
        friend  std::ostream&
            operator <<(std:: ostream & os,String   const& s)   // тут оператор сможет извне класса обращаться к приватным
        {
            return  os << s.data_;
        }
    private:
        char * data_;
        size_t  len_;
    };

## <a id='toc2_3_'></a>[Дружественные методы](#toc0_)

Частный случай дружественной функции. Конкретному методу конкретного класса разрешается извне работать с приватными полями.

    struct  String;                                             // тот случай, когда надо объявлять несколько раз
    struct  StringBuffer {
        void  append(String  const& s);                         // чтоб тут можно было объявлять вот так
        void  append(char  const* s) {...}
        ...
    };
    
    struct  String {
        ...
        friend void  StringBuffer::append(String  const& s);   // для этого уже должен быть определен класс StringBuffer
    };
    
    // в такой конфигурации мы можем определить метод только снаружи, когда определен String
    // а т.к. определение метода выносится из класса,
    // то если нам надо разместить его в заголовочном файле, нам надо напрямую дописать inline
    void  StringBuffer::append(String  const& s) {
        append(s.data_);
    }

## <a id='toc2_4_'></a>[Отношение дружбы](#toc0_)

Отношение дружбы можно охарактеризовать следующими утверждениями:  

∙Отношение дружбы не симметрично.  
∙Отношение дружбы не транзитивно.  
∙Отношение наследования не задаёт отношение дружбы.  
∙Отношение дружбы сильнее, чем отношение наследования.  

Вывод  
Стоит избегать ключевого слова friend, так как оно нарушает инкапсуляцию.

# <a id='toc3_'></a>[Шаблон проектирования Singleton](#toc0_)

## <a id='toc3_1_'></a>[Класс Singleton](#toc0_)

Класс у которого можно реализовать только 1 экземпляр (например класс для базы данных)

    struct  Singleton {
        static  Singleton & instance () {               // стат.метод (имеет доступ к приватным, и доступен без объекта класса), в котором
            static  Singleton s;                        // объявлена единственная стат. переменная (живет до конца программы) для экземпляра этого класса
            return s;                                   // для работы с ней возвращаем ее по ссылке
        }
        Data & data() { return  data_; }
    private:                                            // запретить внешнему коду создавать экземпляры
        Singleton () {}                                 // конструктор по умолчанию (и другие возможные конструкторы)        
        Singleton(Singleton  const &);                  // объявить но не определять констр.копирования
        Singleton& operator =( Singleton  const &);     // и оператор присваивания
        Data  data_;
    };

Возврат по ссылке - единственный вариант (возвр.по значению вызало бы констр.копирования). Плюс ссылка удобна тем, что за ней не надо следить с точки зрения динамической памяти.    
Метод instance таким образом при первом обращении инициализирует экземпляром класса стат.переменную, этот же метод при последующих обращениях просто возвращает ссылку на нее.  

*) для многопоточности этого недостаточно, применяется ряд хитростей, но идея та же

## <a id='toc3_2_'></a>[Использование Singleton-а](#toc0_)

    int  main()
    {
        // первое обращение
        Singleton & s = Singleton::instance ();
        Data d = s.data ();
        
        // аналогично d = s.data();
        d = Singleton::instance().data();
        return  0;
    }